In [ ]:
!pip install sumeval

In [ ]:
!pip install textstat
import textstat
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
 # this gives us a list of sentences
from gensim.utils import tokenize

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

!pip install simpletransformers
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs
from sumeval.metrics.rouge import RougeCalculator
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/prepocessedAll.csv', delimiter=',',
                 engine='python', error_bad_lines=False, nrows=3000)

print(df.info())

drop_cols = ['title']
df = df.drop(drop_cols, axis=1)
df = df.dropna()

df.rename(columns={"headline":"target_text", "text": "input_text"}, inplace= True)
print(df.info())

print(df.head(1)['input_text'])

In [ ]:
stop_words = set(stopwords.words('english'))

# df_rp = pd.read_csv('data.csv')
# df_rp = df_rp.iloc[: , 1:]


def readScore(text):
  score = textstat.flesch_reading_ease(text)
  return score

model_args = Seq2SeqArgs()
model_args.num_train_epochs = 3
model_args.no_save = True
# model_args.per_device_train_batch_size=16,  # batch size per device during training
# model_args.per_device_eval_batch_size=64,   # batch size for evaluation
model_args.logging_dir='./logs',            # directory for storing logs
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.max_length = 600
model_args.overwrite_output_dir = True

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=False,
)

train, test = train_test_split(df, test_size=0.2, random_state=23, shuffle=True)

model.train_model(train, eval_data=test)

results = model.eval_model(test)

def bartSumm(text):
  strList = [text]
  new = model.predict(strList)
  return new

import pickle

with open("fineTunedBert.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
!pip install datasets transformers huggingface_hub sentencepiece

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
X = df['input_text']
y = df['target_text']

In [ ]:
train,test = train_test_split(df, test_size=0.2, random_state = 1, shuffle=True)


In [ ]:
training = Dataset.from_pandas(pd.concat([X_train, y_train], axis=1), preserve_index=False)
testing = Dataset.from_pandas(pd.concat([X_test, y_test], axis=1), preserve_index=False)

In [ ]:
dataset = DatasetDict({'train': training,'test': testing})

In [ ]:
from transformers import  AutoModelForSeq2SeqLM, BartModel

In [ ]:
from transformers import BartTokenizer, DataCollatorWithPadding
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
def tokenize_function(data):
    return tokenizer(data["input_text"], padding="max_length", truncation=True)

In [ ]:
tokenize_df = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = BartModel.from_pretrained("facebook/bart-large-cnn")

In [ ]:
from transformers import pipeline 


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
summaries=[]
for index,row in test.iterrows():
  summaries.append(summarizer(row['input_text'],max_length=70, min_length=20))

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['summary'])

In [ ]:
import rouge


In [ ]:
rouge_scorer = rouge.Rouge()
scores = rouge_scorer.get_scores(summary_df['summary'], test['target_text'], avg=True)
scores

In [ ]:
rouge_calc(pred, list(test['target_text']))